In [ ]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from geopandas import plotting
##from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons
from matplotlib.animation import FuncAnimation

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual,interactive_output, VBox, HBox 

In [ ]:
os.getcwd()

In [ ]:
path2file = '/home/jupyterlab/data/LSOAs_latest.csv'

In [ ]:
lsoa_latest_df = pd.read_csv(path2file)

In [ ]:
lsoa_latest_df

In [ ]:
all_columns_list = list(lsoa_latest_df.columns)

In [ ]:
id_vars_list = ['lsoa11_cd','lsoa11_nm']
for x in id_vars_list:
    all_columns_list.remove(x)

In [ ]:
all_columns_list

In [ ]:
lsoa_data = pd.melt(lsoa_latest_df, id_vars=id_vars_list, value_vars=all_columns_list)

In [ ]:
lsoa_data

In [ ]:
lsoa_trend_data = lsoa_data.replace(-99,0)
lsoa_trend_data['weekasnumber'] = lsoa_trend_data['variable'].str[3:5].astype(int)

In [ ]:
lsoa_southwark = lsoa_trend_data[lsoa_trend_data.lsoa11_nm.str.contains('Southwark')]
lsoa_lookup_southwark_wk5 = lsoa_southwark[lsoa_southwark['weekasnumber']==15]
lsoa_lookup_southwark_wk5.head(5)

In [ ]:
lsoa_plot = lsoa_southwark.pivot(index='weekasnumber', columns='lsoa11_nm', values='value')
#lsoa_plot.plot()

In [ ]:
import datetime
datetime.date(2020, 10, 8).isocalendar()[1]

In [ ]:
path2file = '/home/jupyterlab/data/Output_Area_to_LSOA_to_MSOA_to_Local_Authority_District__December_2017__Lookup_with_Area_Classifications_in_Great_Britain.csv'

In [ ]:
oa_lookup_df = pd.read_csv(path2file)

In [ ]:
oa_lookup_df.head(5)

In [ ]:
localauthority_list = list(oa_lookup_df['LAD17NM'].unique())
localauthority_list.sort()
localauthority_list

In [ ]:
region_list = list(oa_lookup_df['RGN11NM'].unique())
region_list.sort()
region_list

In [ ]:
oa_lookup_df.columns

In [ ]:
lsoa_lookup_columns = oa_lookup_df[['LSOA11CD', 'LSOA11NM',
        'MSOA11CD', 'MSOA11NM', 'LAD17CD', 'LAD17NM', 'LACCD',
       'LACNM', 'RGN11CD', 'RGN11NM', 'CTRY11CD', 'CTRY11NM']]

In [ ]:
lsoa_lookup_columns.shape

In [ ]:
lsoa_lookup_df = lsoa_lookup_columns.drop_duplicates()

In [ ]:
lsoa_lookup_df.shape

In [ ]:
lsoa_lookup_df.head(5)

In [ ]:
fp = '/home/jupyterlab/data/Lower_Layer_Super_Output_Areas__December_2011__Boundaries_EW_BFC_v3.shp'

In [ ]:
map_df = gpd.read_file(fp)
# check data type so we can see that this is not a normal dataframe, but a GEOdataframe
map_df.head()

In [ ]:
##data2plot1_df = lsoa_lookup_southwark_wk5.merge(lsoa_lookup_df, left_on='lsoa11_cd', right_on='LSOA11CD')
data2plot1_df = lsoa_trend_data.merge(lsoa_lookup_df, left_on='lsoa11_cd', right_on='LSOA11CD')


In [ ]:
data2plot2_df = map_df.merge(data2plot1_df,right_on='lsoa11_cd', left_on='LSOA11CD')

In [ ]:

#['lsoa11_cd', 'lsoa11_nm', 'variable', 'value', 'weekasnumber','Shape__Are', 'Shape__Len', 'geometry']
data2plot_df = data2plot2_df[['lsoa11_cd', 'lsoa11_nm','LAD17NM','RGN11NM','Shape__Are', 'Shape__Len', 'geometry','weekasnumber','value']].copy()
data2plot_df['OBJECTID'] = data2plot_df.index
data2plot_df['cumulative_value'] = data2plot_df.groupby(['lsoa11_nm','weekasnumber'])['value'].apply(lambda x: x.cumsum())
data2plot_df.head(5)

In [ ]:
def filterframe(WeekIn,RegionIn):
    #region_in = 'London'
    lsoa_region = data2plot_df[data2plot_df['RGN11NM'] == RegionIn]
    lsoa_region_week = lsoa_region[lsoa_region['weekasnumber']==WeekIn]
    return lsoa_region_week
#sw_df = filterframe(5,'London')
#sw_df


In [ ]:
#interact(filterframe, WeekIn=widgets.IntSlider(min=5, max=40, step=1, value=25));

In [ ]:
#gdf = GeoDataFrame(data2plot_df)

#data2plot_df.plot(column='value')


In [ ]:
def f(x):
    return x

In [ ]:
interact(f, x=10);

In [ ]:
def plotthegraph(WeekIn,RegionIn):
    data2plot = filterframe(WeekIn,RegionIn)
    fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    ax.set_title(WeekIn)

    data2plot.plot(column='cumulative_value', ax=ax, legend=True)
#plotthegraph(5,'London')

In [ ]:
a = widgets.IntSlider()
b = widgets.IntSlider()
c = widgets.IntSlider()
ui = widgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

In [ ]:
#weekinwidget = widgets.IntSlider(min=5, max=40, step=1, value=25,description='WeekIn')


regionwidget = widgets.SelectMultiple(
    options=region_list,
    value=['London'],
    #rows=10,
    description='RegionIn',
    disabled=False
)

playwidget = widgets.Play(
    value=25,
    min=0,
    max=35,
    step=1,
    interval=100,
    description="Press play",
    disabled=False
)

#widgets.jslink((play, 'value'), (slider, 'value'))
#widgets.HBox([play, slider])

In [ ]:
previousdf = []

In [ ]:


def plotthegraph1(WeekIn,RegionIn):
    #data2plot = filterframe1(WeekIn,Region)
    lsoa_region = data2plot_df[data2plot_df['RGN11NM'].isin(RegionIn)]
    lsoa_region_week = lsoa_region[lsoa_region['weekasnumber']==WeekIn]
    #fig, ax = plt.subplots(1, 1, figsize=(15, 10))
    #lsoa_region_week.plot(column='value', ax=ax, legend=True)
    #lsoa_region_week.plot(df=lsoa_region_week,column='value', ax=ax, legend=True)
    #custom_plot(lsoa_region_week)
    
    #if lsoa_region_week.empty:
    #    plotting.plot_dataframe(previousdf,column='value',
    #                        legend=True,figsize=(10,10), vmin=0, vmax=30,aspect='equal')
    #else:
    
    plotting.plot_dataframe(lsoa_region_week,column='cumulative_value',
                            legend=True,figsize=(10,10), vmin=0, vmax=20,aspect='equal')
    #previousdf = lsoa_region_week

def f(WeekIn,RegionIn):
    print((WeekIn,RegionIn))
    #plotthegraph(5,'London')

w = interactive_output(plotthegraph1,{"WeekIn":playwidget,"RegionIn":regionwidget})

hbox1 = HBox([playwidget,regionwidget])
#hbox2 = HBox([measurewidget,indexwidget, activitywidget])
vbox =  VBox([hbox1])

display(w,vbox)

#out = widgets.interactive_output(f, {"WeekIn":weekinwidget,"RegionIn":regionwidget})

#display(ui, out)

In [ ]:
#np.linspace(1, 40, 40).tolist()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 10))

def init():
    data2plot = filterframe(5,'London')
    #plt.plot(data2plot,ax=ax, legend=True)
    #ax = custom_plot(data2plot)
    ax_r = plotting.plot_dataframe(lsoa_region_week,ax=ax)
    return ax_r

def update(frame):
    data2plot = filterframe(Frame,'London')
    #ax = custom_plot(data2plot)
    ax_r = plotting.plot_dataframe(lsoa_region_week,ax=ax)
    return ax_r

ani = FuncAnimation(fig, update, frames=np.linspace(1, 40, 40),
                    init_func=init)
plt.show()